<a href="https://colab.research.google.com/github/gvogiatzis/CS3320/blob/main/CS3320_Lab_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS3320 Lab 2. Ranked Retrieval

## Introduction
In this lab we will further explore our **BBC Sports** dataset by applying some ranked retrieval algorithms. In the process we will look at TF-IDF scoring and perform some queries with free text. Our search engine should use the following formula for the tf-idf weightings

$$tfidf_{t,d}=tf_{t,d}\times\log_{10}\left(\frac{N}{df_{t}}\right)$$ 

where $tf_{t,d}$ is the number of times term $t$ appears in document $d$ and $df_{t}$ is the number of documents that contain term $t$ and $N$ is the total number of documents in the collection. We should also normalize your documents but not the query. 

Let's download the files again and unzip them in the google colab virtual computer drive.

In [ ]:
!wget https://github.com/gvogiatzis/CS3320/raw/main/data/bbc_sport_docs.zip
!unzip bbc_sport_docs.zip -d docs

Let's import some toolboxes again.

In [ ]:
import re
import os
from glob import glob
from math import log10

We re-run the code from the previous lab that reads the files and does the tokenization. None of that needs to change.

In [ ]:
def readfile(fname):
    f = open(fname, 'r', encoding='latin-1')
    s = f.read()
    f.close()
    return s

def tokenize(text):
    DELIM = '[ \r\n\t0123456789;:.,/\(\)\"\'-]+'
    return re.split(DELIM, text.lower())

## Term frequencies

In the Boolean Retrieval model (Lab 01) we saw how the postings datastructure was essentially a mapping from terms in the dictionary, to the document id's that contain that term. A mapping of this type can elegantly be represented in Python using a *dictionary*. So a postings dictionary such as
    
    {'cricket': {2, 3, 5, 7}, 'football': {0, 2, 4}, 'rugby': {1, 2, 6}}
    
would denote that the word *cricket* can be found in documents with id's 2, 3, 5 and 7 etc. 

Now the only trouble is that for ranked retrieval we need the *number of times* a particular term appears in a particular document and the postings structure described above does not provide that information. What can we do instead? 

For ranked retrieval we would need a datastructure which maps a term and a document to a number, i.e. how many times that term is contained in that document. This could be achieved by something like:

    {('cricket',2) : 5, ('cricket',3) : 10, ('football',4):20, ('football',5):30, ('football',6):40}

However we can achieve the same thing much more efficiently by a double-mapping from term to document to number. Something like the following:

    {'cricket':{2:5, 3:10}, 'football':{4:20,5:30,6:40}}

This is essentialy a dictionary with the terms being the keys, and the items being dictionaries themselves. These second-level dictionaries map document id's to the number of times the term appears in them. 

Let's say we gather a list of all the docIDs where that term appears. Note this is a list that may have duplicates arising from the same term appearing multiple times in the same document. E.g. [2,2,2,3,3,3,3,4,4]. 

To summarise this list using document counts, we create an empty dictionary. Thenwe go through that list of docIDs and everytime we come across a docID we havent seen before, we set the entry to one. If a docID entry exists we increment it by one. 
The result should be  {2:3, 3:4, 4:2} for the list above. This dictionary says that docid 2 has been found 3 times, docid 3 has been found 4 times and docid 4 has been found 2 times. This is what that looks like in Python:


In [ ]:
a = [2,2,2,3,3,3,3,4,4]

c = dict()

for x in a:
    if x in c:
        c[x] += 1
    else:
        c[x] = 1

print(c)

This code can be simplified somewhat by using a special datastructure known as a *defaultdict*. The difference to a normal dict is that any key that hasn't been given an entry yet, is assumed to have a default value (in our case 0). Using a defaultdict the code above would look like

In [ ]:
from collections import defaultdict

a = [2,2,2,3,3,3,3,4,4]

c = defaultdict(int)

for x in a:
    c[x] += 1

print(c)

which is a bit more *pythonic*. Challenge: Python `collections` has a type of data structure object called a `Counter`. Can you use a `Counter` to create the structure above given the array `a`, in a **single line of code**? Just search for `collections.Counter` to figure out the correct usage.

We are now ready to create a function that will generate term frequencies for us. The structure of the code will follow that of the boolean retrieval model.

In [ ]:
def create_tf_structure(path):
    fnames = sorted(glob(path)) # load all filenames
    tf=defaultdict(lambda : defaultdict(int)) 
    for docID,fname in enumerate(fnames):
        s = readfile(fname)
        words = tokenize(s)
        for w in words:
            tf[w][docID] += 1    
    return tf

Let's see this in action:

In [ ]:
tf = create_tf_structure("docs/*.txt")

Use the box below, to print out the postings for the word "incredible".

## Inverse document frequencies

Our next task is to compute the second component of TF-IDF which is the Inverse Document Frequency. And to do that, a good idea would be to start with Document Frequency. I.e. for each word, the number of documents that contain it. Luckily this can be computed from the `tf` structure computed above. You see for a term `w`, the dictionary `tf` has exactly as many elements as the number of documents that contain `w`. So `len[tf[w]]` would give us exactly what we need:

In [ ]:
df = dict()

for w in tf.keys():
    df[w] = len(tf[w])

If you want, you can write this in a single line using Python comprehensions as follows:

In [ ]:
df = {w:len(tf[w]) for w in tf.keys()}

This is briefer and arguably easier to interpret. Now for the IDF formula we require *N* the numbe of all documents. This can be obtained from 

In [ ]:
N = len(glob('docs/*.txt'))

In the box below, please write the one-line python code for computing `idf`. [Hint: modify the `df` code appropriately.

What is the `idf` of the word "unique". Use the box below.

## Document normalization

What's left to do is to compute the normalization factors for each document. These will be used when computing the ranking score. Given the `tf` structure we can loop through it and add all the squares of the term frequencies found in each document. We then need to take the square root of the value accumulated. This can be done as follows: 

In [ ]:
def normalisation(tf):
    norms = defaultdict(float)
    # Loop through all words and their postings lists
    for w, p in tf.items():
        # for each postings list add the square of the term frequency to the entry for that document
        for d,f in p.items():
            norms[d] += f**2
    
    # Just making sure we record the sqrt of the sum of squares.
    for d, n in norms.items():
        norms[d] = sqrt(n)
    return norms

We can now put everything together in an indexer function as follows:

In [ ]:
from math import log10,sqrt

def indextextfiles_RR(path):
    #preparing document norm dictionary
    #counting total number of docs (needed for idf)
    fnames = sorted(glob(path))
    N = len(fnames)
    #preparing and empty dict of dicts to hold term frequencies
    tf=defaultdict(lambda : defaultdict(int))
    # for each word w, tf[w] is the postings dictionary.
    # i.e. a mapping from docID to a number that signifies how many times 
    # word w is contained in document docID.
    
    for docID,fname in enumerate(fnames):
        # read each file into a string
        s = readfile(fname)
        # split string into a list of tokens
        words = tokenize(s)
        # for each word w, increment the corresponding entry for docID
        # i.e. increase by one, the number of times that word w has appeared
        # in document docID.
        for w in (w for w in words if w!=''):
            tf[w][docID] += 1
            
    # idf is a dictionary that maps each word w to log10(N/len(p))
    # where p is the dictionary that maps docIDs to how many times w appears
    # in the corresponding doc. If we write len(p) python will return the number
    # of docs that contain the term w at least once (all the docIDs that have a zero count
    # don't get counted in the len). This is exactly equal to df[w] and we can use it
    # in the idf calculation as per the forula provided above.
    idf = {w:log10(N/len(p)) for w,p in tf.items()}
    
    norms = normalisation(tf)
    # Returning the postings, idf and norms dictionaries.
    return tf, idf, norms

Let's apply this to our data folder:

In [ ]:
tf, idf, norms = indextextfiles_RR("docs/*.txt")

## Evaluating a query

We are now ready to start evaluating some queries. Remember, for each term `t` and for each document `d` that contains that term, we need to add  $tfidf_{t,d}$ weight to that document's score. The code below explains the whole process.

In [ ]:
def query_RR(tf,idf,norms, qtext, K):
    # We split the query text into tokens using identical processing as
    # for the indexing stage
    words = tokenize(qtext)
    #initialize an empty dictionary to hold the score per document
    scores = defaultdict(float)
    # for each word in query, retrieve its postings
    for w in words:
        # for each document containing the query word, add its term frequency
        # multiplied with the idf weight of the query word.
        for d,TF in tf[w].i tems():
            scores[d] += idf[w]*TF


    # make sure we normalize the score of each document by dividing it
    # by the norm of that document
    for d,s in scores.items():
        scores[d] = s/norms[d]
    # Sort the results in reverse order (big to small)
    # The sorted function when applied to a dictionary normally returns
    # th dictionary keys, sorted. By supplying the key function as res.get
    # we are forcing it to sort the dictionary by the item mapped by each key.
    rs =sorted(res,key=res.get, reverse=True)
    # filter out all but the top K scores
    rs = rs[0:K]
    # return the indices of the
    return rs, [res[r] for r in rs]

Use the code given above to obtain the 10 most relevant documents to the query "england played very well". Write your answer to the box below.

The code provided in the box below is given a string `s` and a word `q` and returns an annotaded text that has got the word `q` highlighted. To see the annotation you need to print the annotated string using the `print` command.

In [ ]:
def highlighttext(s, q):
    RED = "\033[1;30;43m"
    RESET = "\033[0;0m"
    REVERSE = "\033[;7m"
    words = tokenize(s)
    qwords = tokenize(q)
    hwords = ['**'+RED+w.upper()+RESET+'**' if w in qwords else w for w in␣
    ,→words]
    return " ".join(hwords)

Use `highlighttext` together with the code shown in the previous lab for loading and printing text files in your doc folder. You can then verify that the query terms you used above do indeed appear in the returned documents. Place your answer in the box below:

Please save this as pdf by going to File->Print and select destination as "Save to pdf". You can then upload the file to blackboard using the submission link found next to the lab spec.